# Utils



In [ ]:
import sys

if 'google.colab' in str(get_ipython()):
    from google.colab import files
    !rm -rf github && mkdir github && cd github && git clone --branch main https://github.com/engdoreis/OTRegressionReportTools.git && echo $pwd
    sys.path.append('github/OTRegressionReportTools')

import IPython
import ipywidgets as widgets
from IPython.display import HTML
from OT_regression_report_parser import *

# Configs

In [ ]:

pd.set_option('display.max_rows', 400)
class Config:
  black_list_days = ['2022-10-28', '2022-10-29', '2022-10-30', '2022-10-31']
  max_reports = 15                        # The amount of older reports should be processed.
  healthy_executions_threshold = 5        # The amount of successful execution will be considered healthy and filtered out.
  hide_healthy_tests = True               # Hide test considered healthy according to `healthy_executions_threshold`.
  hide_empty_tests = True                 # Hide not implemented tests.
  invert_columns_order = True             # False : The latest will show in the right.
  sort_tests_by_failure = True            # Show the test failing on the top.
  attach_github_issues = True             # Search and link opened github issues.
  add_build_data = False                  # Add github revision and build seed.
  hide_passrate = False                   # Hide the cells content and show the background color only.
  base_url = 'https://reports.opentitan.org/hw/top_earlgrey/dv/'
report = RegressionReportParser(Config())

# Test results

In [ ]:
test_report = report.generate_test_results_report()
HTML(report.format_dataframe(test_report).render())

In [ ]:
open('report.html', 'w').write(report.format_dataframe(test_report).render())
btn = widgets.Button(description='Download report')
btn.on_click(lambda x: files.download('report.html'))
btn

# Progress

In [ ]:
test_progress = report.generate_testplan_progress_report()
display(test_progress)
test_progress.transpose().plot.bar(figsize=(38,6),rot=45)

# Pareto


In [ ]:
err = report.generate_errors_report()
display(err)

In [ ]:
ax = err.plot(figsize=(32,6), kind='bar', y='Error%', color='darkred')
err.plot(figsize=(32,6), kind='line',  y='cumError%', grid=True, color='darkblue', ax=ax)

# List of issues that can be closed
TODO

# Tests ramp up


In [ ]:
df = report.generate_tests_rampup(test_report)
fig, ax = plt.subplots(figsize=(32,12))
df.plot(ax=ax, kind='line', grid=False, ylabel='Amount of tests', xlabel='days', x='Date', marker='x').legend(loc='upper center')
df.iloc[0::len(df.index)-1,:].apply(lambda row: ax.annotate(f'{row.Count} [{row.Date.strftime("%d/%b")}]', xy=(row.Date, row.Count-1), fontsize=14, rotation=0, color='darkred') ,axis=1)
ax.annotate(f'Increase of {round((df.Count.iloc[-1]/df.Count.iloc[0]-1)*100,1)}%', xy=(df.Date.iloc[0], df.Count.iloc[-2]), fontsize=16, color='darkred')
plt.show()

# Status by Test
This cell is intented to help filling the github issues with a nice table before closing it. The output is a markdown which can be copied and pasted in the github issue.
The search accepts partial matching and multiple tests can be matched using `|` (or) pattern.

In [ ]:
print(report.generate_test_results_report(by='chip_csr_mem_rw_with_rand_reset timeout').to_markdown())